# Identify the impact of an event in a particular area  


> 👋 Before moving on with this demo, you must first sign-up and request your Geosys APIs credentials here :
> - ⚙️[Try it now](https://www.earthdaily.com/geosys/geosys-api/)

> For more information about our Geosys APIs : 
> - 📚 [Geosys APIs to connect with your digital ag application](https://www.earthdaily.com/geosys/geosys-api/)


> **Demo Project:** This demo demonstrates the ability to identify and calculate the impacted area within a given geometric region resulting from a specific event.



### @author: Geosys



 ## 1️⃣ Import all librairies needed and get an autorization to use ImpactedAreasIdentificator

In [ ]:
import sys
sys.path.append('../src/')
from geosyspy import Geosys
from dotenv import load_dotenv
from dateutil.relativedelta import relativedelta
from matplotlib.colors import LinearSegmentedColormap,ListedColormap
from geosyspy.utils.constants import *
from vegetation_index_impacted_areas_identificator.impacted_areas_identification import *

import logging
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np
import datetime as dt
import os

logger = logging.getLogger()
logger.setLevel(logging.INFO)

# read .env file
load_dotenv()

API_CLIENT_ID = os.getenv('API_CLIENT_ID')
API_CLIENT_SECRET = os.getenv('API_CLIENT_SECRET')
API_USERNAME = os.getenv('API_USERNAME')
API_PASSWORD = os.getenv('API_PASSWORD')

client = ImpactedAreasIdentificator(API_CLIENT_ID, API_CLIENT_SECRET, API_USERNAME, API_PASSWORD, Env.PROD, Region.NA)


 ## 2️⃣ Input data

These input parameters are utilized in the demo to identify and calculate the impacted area resulting from the specified event and threshold within the given geometric region. The input data for this example consists of the following parameters:

##### polygon: 
A polygon string in WKT format representing the geometric region of interest. This polygon defines the boundaries of the area under analysis.

#### event_date: 
A datetime object representing the date of the event. In this case, the event date is "2021-06-07".

#### threshold: 
A floating-point value representing the threshold for evaluating the impact. In this example, the threshold is set to -0.15.




In [ ]:
polygon = "POLYGON((-55.08964959 -12.992576790000001, -55.095571910000004 -12.99349674, -55.09265364 -13.014153310000001, -55.07111016 -13.01013924, -55.07428588 -12.98914779, -55.08261147 -12.99098782, -55.08115233 -13.00152544, -55.08724632 -13.00269622, -55.08819045 -13.0037834, -55.08956371 -13.00369981, -55.08819045 -13.00202724, -55.08964959 -12.992576790000001))"

event_date = dt.datetime.strptime("2021-06-07", "%Y-%m-%d")

threshold = -0.15

## 3️⃣ Call the coverage for the area and retrieve images before and after the event date

To obtain coverage information for the area of interest, make a call to retrieve satellite images captured before and after the event date.

In [ ]:
coverage_info_df, images_references = client.get_image_coverage_info_based_on_map_reference(polygon, event_date)

image_date_list = pd.to_datetime(coverage_info_df['image.date']).dt.date

## 4️⃣  Retrieve ndvi images before and after to the specified event date 

Retrieve the NDVI (Normalized Difference Vegetation Index) images captured before and after the specified event date


In [ ]:
nearest_event_date = client.find_nearest_dates(event_date.date(), image_date_list)

ndvi_images_nearest_event_date = client.get_ndvi_image_time_series(polygon,
                                                                         nearest_event_date["before_event_date"],
                                                                         nearest_event_date["after_event_date"])

ndvi_before_event_date = ndvi_images_nearest_event_date.sel(time=str(nearest_event_date["before_event_date"]))['ndvi']

ndvi_after_event_date = ndvi_images_nearest_event_date.sel(time=str(nearest_event_date["after_event_date"]))['ndvi']

## 5️⃣Retrive the impacted area based on a threshold

Calculate and retrieve the impacted area based on a specified threshold value.


In [ ]:
ndvi_impacted_area = client.calculate_and_filter_ndvi_difference_by_threshold(ndvi_before_event_date,
                                                                                  ndvi_after_event_date,
                                                                                  threshold)

### Retrieve the impacted area (area and %)

In [ ]:
impacted_area,impacted_area_percentage = client.calculate_impacted_area(polygon, ndvi_impacted_area)

print('# Impacted area: {:12.3f} m²'.format(impacted_area))
print('# Impacted area percentage: {:2.2f} %'.format(impacted_area_percentage))

### Retrieve the impacted area (xarray)

In [ ]:
ndvi_impacted_area

##  6️⃣ Display the results
Visualize the results using matplotlib by displaying the NDVI of the image just before the event date, the image after the event date, and the impacted area.

In [ ]:
# Select only x and y dimension to display ndvi
ndvi_before_event_date_image= ndvi_before_event_date.squeeze().drop(['time', 'band'])
ndvi_after_event_date_image= ndvi_after_event_date.squeeze().drop(['time', 'band'])

fig, ax = plt.subplots(1, 2, figsize=(10,4))
colors =['red', 'yellow', 'green']
cmap =  LinearSegmentedColormap.from_list(['red_green'],colors)
ndvi_before_event_date_image.plot(ax=ax[0], cmap = cmap)
ax[0].set_title(f"{np.datetime_as_string(ndvi_before_event_date['time'].values, unit='D')}, {ndvi_before_event_date['image.sensor'].values}")

ndvi_after_event_date_image.plot(ax=ax[1],  cmap = cmap)
ax[1].set_title(f"{np.datetime_as_string(ndvi_after_event_date['time'].values, unit='D')}, {ndvi_after_event_date['image.sensor'].values}")

fig.tight_layout()

fig2, ax2 = plt.subplots()
plt.imshow(ndvi_after_event_date_image,  cmap = cmap, alpha = 0.05)
plt.imshow(ndvi_impacted_area, aspect = 'auto')
plt.title(f"NDVI threshold= {threshold}")
plt.contourf(ndvi_impacted_area, colors = 'red', alpha = 0.7)
plt.legend(handles=[plt.Rectangle((0,0),1,1, color = 'red')], labels=['Impacted Area'])